In [1]:
import pandas as pd
import requests,re
import pickle

In [2]:
# 计算两点间距离
def compute_distance(longitude1,latitude1,longitude2,latitude2):
    header = {'user_agent':'Mozilla/5.0 (Windows NT 10.0;WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    request_url = 'http://restapi.amap.com/v3/distance?key=6c4e7caf822a3add82fdd0a71c4258e9&origins='\
    +str(longitude1) + ',' + str(latitude1) + '&destination=' + str(longitude2) + ',' + str(latitude2) +'&type=1'
    data = requests.get(request_url,headers = header,timeout = 100)
    data.encoding = 'utf-8'
    data = data.text
    #"distance":"2646","duration":"660"
    pattern = '"distance":"(.*?)","duration":"(.*?)"'
    result = re.findall(pattern,data)
    #返回距离result[0][0],返回时间result[0][1]
    return result[0][0]

In [3]:
#通过keyword,city找到指定的location
def get_location(keyword,city):
    request_url = 'http://restapi.amap.com/v3/place/text?key=6c4e7caf822a3add82fdd0a71c4258e9&keywords='+keyword+'&types=&city='+city+'&children=1&offset=1&page=1&extensions=all'
    header = {'user_agent':'Mozilla/5.0 (Windows NT 10.0;WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    data = requests.get(request_url,headers = header,timeout = 100)
    data.encoding = 'utf-8'
    data = data.text    
    #.*表示贪婪模式，匹配到不能匹配为止
    #.*?取消贪婪，一个匹配后就继续后面的匹配
    pattern = '"location":"(.*?),(.*?)"'
    result = re.findall(pattern,data)
    try:
        return result[0][0],result[0][1]
    except:
        return get_location(keyword.replace('站',''),city)

In [4]:
# 数据加载
def load_data():
    data = pd.read_csv('./subway.csv',index_col = None)
    return data

In [5]:
# 计算并保存两点间距离,保存python对象，方便下次调用.如已计算则直接调用
def generate_graph():
    try:
        file = open('graph.pkl','rb')
        graph = pickle.load(file)
        return graph
    except:
        from collections import defaultdict
        # 创建两点间最短距离
        data = load_data()
        graph = defaultdict(dict)
        for i in range(data.shape[0]-1):
            site1 = data.iloc[i]['site']    
            site2 = data.iloc[i+1]['site']   
            #如果是同一条线路
            if site1 == site2:
                name1,name2 = data.iloc[i]['name'],data.iloc[i+1]['name']
                longitude1,latitude1 = data.iloc[i]['longitude'],data.iloc[i]['latitude']
                longitude2,latitude2 = data.iloc[i+1]['longitude'],data.iloc[i+1]['latitude']
                distance = compute_distance(longitude1,latitude1,longitude2,latitude2)
                graph[name1][name2] = distance
                graph[name2][name1] = distance
                #print(name1,name2,distance)   
        #print(len(graph))
        output = open('graph.pkl','wb')
        pickle.dump(graph,output) 
        return graph

In [6]:
# 找到最小开销的节点
def find_lowest_cost_node(costs):
    #初始化数据
    lowest_cost = float('inf')
    lowest_cost_node = None
    for node in costs:
        #如果节点没有被处理
        if not node in processed:
            #如果当前的node比已经存在的node的coss小，更新数据
            if costs[node] < lowest_cost:
                lowest_cost = costs[node]
                lowest_cost_node = node
    return lowest_cost_node

In [7]:
# 找到最短路径,从start到end的最短路径
def find_shortest_path():
    node = end
    shortest_path = [end]
    while parents[node] != start:
        shortest_path.append(parents[node])
        node = parents[node]
    shortest_path.append(start)
    return shortest_path[::-1]    

In [14]:
# 计算从start到end的最短路径
def dijkstra():
    graph = generate_graph()
    #查询到目前开销最小的节点
    node = find_lowest_cost_node(costs)
    #print('当前cost最小节点',node)
    #只要有cost最小的节点就进行路径计算，如果所有节点都在processed中结束计算。
    while node is not None:
        #获取节点目前的cost
        cost = costs[node]
        #获取节点的邻居节点
        neighbors = graph[node]
        #遍历所有邻居节点，看是否可以更新cost
        for neighbor in neighbors.keys():
            #计算经过当前节点到达邻居节点的cost
            new_cost = cost + float(neighbors[neighbor])
            if neighbor not in costs or new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                parents[neighbor] = node
        #将当前节点标记为处理过
        processed.append(node)
        #找出下一步要处理的节点
        node = find_lowest_cost_node(costs)
    #循环完毕完成，说明所有节点处理完毕
#     shortest_path = find_shortest_path()
#     print('从{}到{}的最短路径为:\n{}'.format(start,end,shortest_path))
#     print('最短距离为：',costs[end])
#     print('途径站点数：',len(shortest_path))

In [15]:
# 找到最近的地铁站点
def get_nearest_subway(data,location1):
    distance = float('inf')
    nearest = None
    for i in range(data.shape[0]):
        site1 = data.iloc[i]['name']
        longitude = float(data.iloc[i]['longitude'])
        latitude = float(data.iloc[i]['latitude'])
        #计算距离
        temp = (longitude - float(location1[0]))**2 + (latitude - float(location1[1]))**2
        if temp < distance:
            distance = temp
            nearest = site1
           
    return nearest    

In [16]:
# 两点间路径规划
def route_planning(site1,site2,city = '北京'):
    global start,end,costs,parents,processed
    graph = generate_graph()
    location1 = get_location(site1,city)
    #print(location1)
    location2 = get_location(site2,city)
    #print(location2)
    data = pd.read_csv('./subway.csv')
    #print(data.head())
    #获得距起点和终点最近的地铁站
    start = get_nearest_subway(data,location1)
    #print(start)
    end = get_nearest_subway(data,location2)
    #print(end)
    print('离起点{}最近的地铁站为：{},离终点{}最近的地铁站为：{}'.format(site1,start,site2,end))
    # 创建节点开销表，costs指节点到start的距离
    costs = {}
    # 存储父节点的hash表，用于记录路径
    parents = {}
    parents[end] = None
    # 记录处理过的节点的列表
    processed = []
    # 获取节点的邻居节点
    for node in graph[start].keys():
        costs[node] = float(graph[start][node])
        parents[node] = start
    costs[end] = float('inf')
    dijkstra()
    #得到最短路径
    shortest_path = find_shortest_path()
    #如果起止点不是地铁站，在路径中增加起止点
    if site1 != start:
        shortest_path.insert(0,site1)
    if site2 != end:
        shortest_path.append(site2)
    print('从{}到{}的最短路径为:\n{}'.format(site1,site2,shortest_path))
    print('最短距离为：',costs[end])
    print('途径站点数：',len(shortest_path))

In [17]:
if __name__ == '__main__':
    site1 = '五道口站'
    site2 = '北京南站'
    route_planning(site1,site2)
    site3 = '良乡大学城站'
    route_planning(site1,site3)
    site4 = '雍和宫站'
    site5 = '西二旗站'
    route_planning(site4,site5)
    site6 = '清华大学'
    site7 = '798'
    route_planning(site6,site7)

离起点五道口站最近的地铁站为：五道口站,离终点北京南站最近的地铁站为：北京南站
从五道口站到北京南站的最短路径为:
['五道口站', '知春路站', '大钟寺站', '西直门站', '新街口站', '平安里站', '西四站', '灵境胡同站', '西单站', '宣武门站', '菜市口站', '陶然亭站', '北京南站']
最短距离为： 31881.0
途径站点数： 13
离起点五道口站最近的地铁站为：五道口站,离终点良乡大学城站最近的地铁站为：良乡大学城站
从五道口站到良乡大学城站的最短路径为:
['五道口站', '知春路站', '知春里站', '海淀黄庄站', '人民大学站', '魏公村站', '国家图书馆站', '白石桥南站', '白堆子站', '军事博物馆站', '北京西站', '六里桥东站', '六里桥站', '七里庄站', '丰台东大街站', '丰台南路站', '科怡路站', '丰台科技园站', '郭公庄站', '大葆台站', '稻田站', '长阳站', '篱笆房站', '广阳城站', '良乡大学城北站', '良乡大学城站']
最短距离为： 74196.0
途径站点数： 26
离起点雍和宫站最近的地铁站为：雍和宫站,离终点西二旗站最近的地铁站为：西二旗站
从雍和宫站到西二旗站的最短路径为:
['雍和宫站', '安定门站', '鼓楼大街站', '积水潭站', '西直门站', '大钟寺站', '知春路站', '五道口站', '上地站', '西二旗站']
最短距离为： 31312.0
途径站点数： 10
离起点清华大学最近的地铁站为：五道口站,离终点798最近的地铁站为：将台站
从清华大学到798的最短路径为:
['清华大学', '五道口站', '知春路站', '西土城站', '牡丹园站', '健德门站', '北土城站', '安贞门站', '惠新西街南口站', '芍药居站', '望京西站', '望京站', '阜通站', '望京南站', '将台站', '798']
最短距离为： 31378.0
途径站点数： 16
